In [1]:
import os
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
import pandas as pd
from flask_pymongo import PyMongo
import pymongo

In [2]:
url = 'https://mars.nasa.gov/news/'
url2 = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
url3 = 'https://twitter.com/marswxreport?lang=en'
url4 = 'http://space-facts.com/mars/'
url5 = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

In [3]:
response = requests.get(url)
response2 = requests.get(url2)
response3 = requests.get(url3)
response5 =  requests.get(url5)
soup = bs(response.text,'html.parser')
soup2 = bs(response2.text,'html.parser')
soup3 = bs(response3.text,'html.parser')
soup5 = bs(response5.text,'html.parser')

url4tbl = pd.read_html(url4)
url4toHTML = url4tbl[0].to_html()
hem_dict_dict ={}

In [4]:
div_class_slide = soup.find_all('div', class_ = 'slide')
par = div_class_slide[0].find('div',class_='rollover_description_inner').text
title = div_class_slide[0].find('div',class_='content_title').find('a').text

In [5]:
#for e in div_class_slide:
#    rdi = e.find('div', class_='rollover_description_inner')
#    ct = e.find('div', class_='content_title')
#    a = ct.find('a').text

In [6]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless = False)
browser.visit(url2)
featured_image_url = 'https://www.jpl.nasa.gov' + soup2.find('footer').find('a').attrs['data-fancybox-href']
mars_weather = soup3.find_all('div', class_ = 'js-tweet-text-container')[0].find('p').text

In [8]:
browser.visit(url5)
s5 = soup5.find_all('div', class_='item')
for each in s5:
    urls5 = 'https://astrogeology.usgs.gov'+ each.find('a').attrs['href']
    browser.visit(urls5)
    rs5 = requests.get(urls5)
    ps5 = bs(rs5.text,'html.parser')
    allAnchors = ps5.find_all('div', class_='downloads')
    hem_dict_dict.update({'title' : ps5.find('title').text , 'img_url' : allAnchors[0].find('a').attrs['href']})  

In [11]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client.mars_db
db.mars.drop()

# Creates a collection in the database and inserts two documents
db.mars.insert_one({'url':url, 'title': title, 'par': par})
db.mars.insert_one({'url2':url2, 'featured_image_url': featured_image_url})
db.mars.insert_one({'url3':url3, 'mars_weather': mars_weather})
db.mars.insert_one({'url4':url4, 'url4toHTML': url4toHTML})
db.mars.insert_one({'hem_dict_dict':hem_dict_dict})


In [45]:
#fnd = db.mars.find()
#for obj in fnd:
#    print(f'{obj}########################################')
#fnd.prettify()    
    

In [46]:
#fnd = db.mars.find()
#fnd[2]['mars_weather']